In [ ]:
import numpy as np
from catboost import CatBoostRegressor, Pool
from catboost.utils import get_gpu_device_count
import time

print("Number of GPUs visible to CatBoost:", get_gpu_device_count())

# Create dummy data
n_rows = 1_000_000
n_cols = 100
X = np.random.rand(n_rows, n_cols)
y = np.random.rand(n_rows)

pool = Pool(X, y)

params = {
    "iterations": 200,
    "depth": 6,
    "learning_rate": 0.1,
    "task_type": "GPU",
    "devices": "0",
    "verbose": False
}

print("Training on GPU ...")
start = time.time()
model = CatBoostRegressor(**params)
model.fit(pool)
end = time.time()
print(f"GPU training done in {end-start:.2f} seconds")

# For comparison run CPU training
params_cpu = params.copy()
params_cpu["task_type"] = "CPU"
print("Training on CPU ...")
start2 = time.time()
model_cpu = CatBoostRegressor(**params_cpu)
model_cpu.fit(pool)
end2 = time.time()
print(f"CPU training done in {end2-start2:.2f} seconds")


In [ ]:
import time
import numpy as np
from statistics import median
from catboost import CatBoostRegressor, Pool
from catboost.utils import get_gpu_device_count

def bench_once(n_rows=1_000_000, n_cols=100, iters=500, depth=8, lr=0.1):
    X = np.random.rand(n_rows, n_cols)
    y = np.random.rand(n_rows)
    pool = Pool(X, y)

    gpu = CatBoostRegressor(
        iterations=iters,
        depth=depth,
        learning_rate=lr,
        task_type="GPU",
        devices="0",
        verbose=False
    )
    t0 = time.time()
    gpu.fit(pool)
    t1 = time.time()

    cpu = CatBoostRegressor(
        iterations=iters,
        depth=depth,
        learning_rate=lr,
        task_type="CPU",
        verbose=False
    )
    t2 = time.time()
    cpu.fit(pool)
    t3 = time.time()

    return (t1 - t0), (t3 - t2)

def bench(repeats=3, **cfg):
    gpu_ts, cpu_ts = [], []
    for _ in range(repeats):
        g, c = bench_once(**cfg)
        gpu_ts.append(g); cpu_ts.append(c)
    g_med, c_med = median(gpu_ts), median(cpu_ts)
    speedup = c_med / g_med if g_med > 0 else float("inf")
    print(f"GPUs visible: {get_gpu_device_count()}")
    print(f"Config: {cfg}")
    print(f"GPU times (s): {gpu_ts}")
    print(f"CPU times (s): {cpu_ts}")
    print(f"Median GPU: {g_med:.2f}s | Median CPU: {c_med:.2f}s | Speedup: {speedup:.2f}x")

# Example: scale dataset/iterations to see the curve
bench(repeats=3, n_rows=800_000, n_cols=80, iters=600, depth=8, lr=0.1)
